In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path
from time import time, strftime, gmtime
from tqdm import tqdm

import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose, Normalize


import sys
sys.path.append('../')

from loss_functions import *
from train_test import train, test
from noise import AddGaussianNoise

In [2]:
sigma = 0.0   # Standard error for feature noise
eta   = 0.0   # Probability of no change for label noise 

test_size = 0.2

# 100 features
# 10 classes

train_data = torch.load('data-sub/train_data-sigma{}_eta{}'.format(sigma,eta))
test_data = torch.load('data-sub/test_data-sigma{}_eta{}'.format(sigma,eta))

# Create data loaders
batch_size_train = 64
batch_size_test  = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size_train, shuffle=True)
test_dataloader  = DataLoader(test_data,  batch_size=batch_size_test,  shuffle=True)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [3]:
# Create model

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define models
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=300, bias=True)
    (1): ReLU()
    (2): Linear(in_features=300, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=10, bias=True)
  )
)


In [4]:
def run(train_dataloader, test_dataloader, model, loss_fn, optimizer, epochs):

    train_loss0 = []
    train_acc0  = []

    train_loss = []
    train_acc  = []
    test_loss  = []
    test_acc   = []

    tr_ls, tr_ac = test(train_dataloader, model, loss_fn)
    train_loss.append(tr_ls)
    train_acc.append(tr_ac)

    te_ls, te_ac = test(test_dataloader, model, loss_fn)
    test_loss.append(te_ls)
    test_acc.append(te_ac)

    for t in tqdm(range(epochs)):
        #print(f"Epoch {t+1}")

        tr_ls0, tr_ac0 = train(train_dataloader, model, loss_fn, optimizer)
        train_loss0.extend(tr_ls0)
        train_acc0.extend(tr_ac0)  

        tr_ls, tr_ac = test(train_dataloader, model, loss_fn)
        train_loss.append(tr_ls)
        train_acc.append(tr_ac)

        te_ls, te_ac = test(test_dataloader, model, loss_fn)
        test_loss.append(te_ls)
        test_acc.append(te_ac)

        #print(f"Test accuracy: {(100*te_ac):>0.1f}%, Test loss: {te_ls:>8f}")
        
    return train_loss0, train_acc0, train_loss, train_acc, test_loss, test_acc

In [5]:
# Create folder
if not os.path.exists('results-lr/'):
    os.mkdir('results-lr/')
    
def my_MSE_loss(x,y):
    return MSE_loss(x,y,10)

def my_MAE_loss(x,y):
    return MAE_loss(x,y,10)

Loss_fn = [my_MSE_loss, my_MAE_loss, CE_loss, FR_loss, H_loss]
titles  = ['MSE', 'MAE', 'Cross-entropy', 'Fisher-Rao', 'Hellinger']

n_trials = 1
epochs   = 20

# All learning rates for all losses
Lr_all   = [[1, 4] for ii in range(5)]
# One learning rate per loss
#Lr_all   = [[0.1], [0.5], [0.005], [0.01], [0.01]]

# For each experiment (trial)
for i_trial in range(n_trials):
    
    tic = time()
    
    print('\n***Experiment: {}/{}***'.format(i_trial,n_trials))

    # Choose loss function
    for i_loss in range(len(Loss_fn)):
        
        loss_fn = Loss_fn[i_loss]
        print(titles[i_loss])
        
        # Choose learning rate
        for lr in Lr_all[i_loss]:
        
            model = NeuralNetwork().to(device)

            optimizer = torch.optim.SGD(model.parameters(), lr=lr)

            train_loss0, train_acc0, train_loss, train_acc, test_loss, test_acc\
                    = run(train_dataloader, test_dataloader, model, loss_fn, optimizer, epochs)

            filename = 'results-lr/mnist_{}_lr{}_eta{}_trial{}'.format(titles[i_loss],lr,eta,i_trial)
            np.save(filename, [train_loss0, train_acc0, train_loss, train_acc, test_loss, test_acc])
        
    toc = time()
    print('Elapsed time: ' + strftime("%H:%M:%S", gmtime(toc-tic)))


***Experiment: 0/1***
MSE


100%|███████████████████████████████████████████| 20/20 [04:01<00:00, 12.05s/it]
/opt/anaconda/lib/python3.9/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
100%|███████████████████████████████████████████| 20/20 [03:52<00:00, 11.61s/it]


MAE


100%|███████████████████████████████████████████| 20/20 [03:55<00:00, 11.80s/it]


Cross-entropy


...

Fisher-Rao


100%|███████████████████████████████████████████| 20/20 [04:00<00:00, 12.04s/it]


Hellinger


100%|███████████████████████████████████████████| 20/20 [04:01<00:00, 12.07s/it]

Elapsed time: 00:40:48
